In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from pathlib import Path
from ipywidgets import widgets
from collections import Counter

In [2]:
deg_dist_path = Path("../../data/deg_dist/")

In [3]:
plot_df_list = []

for df_path in deg_dist_path.glob('*.csv'):
    N = int(df_path.stem.replace('N_', ''))
    sub_plot_df = pd.read_csv(df_path)
    sub_plot_df['N'] = N
    plot_df_list.append(sub_plot_df)
    
plot_df = pd.concat(plot_df_list)

In [4]:
beta_min = plot_df['beta'].min()
beta_max = plot_df['beta'].max()
beta_step = (beta_max - beta_min) / (plot_df['beta'].nunique()-1)

beta_slider = widgets.FloatSlider(
                            value = beta_max,
                            max = beta_max,
                            min = beta_min,
                            step = beta_step,
                            description = 'beta'
                        )


alpha_min = plot_df['alpha'].min()
alpha_max = plot_df['alpha'].max()
alpha_step = (alpha_max - alpha_min) / (plot_df['alpha'].nunique()-1)

alpha_slider = widgets.FloatSlider(
                            value = alpha_max,
                            max = alpha_max,
                            min = alpha_min,
                            step = alpha_step,
                            description = 'a'
                        )


K_min = plot_df['K'].min()
K_max = plot_df['K'].max()
K_step = (K_max - K_min) / (plot_df['K'].nunique()-1)

K_slider = widgets.FloatSlider(
                            value = K_max,
                            max = K_max,
                            min = K_min,
                            step = K_step,
                            description = 'K'
                        )


N_options = sorted(plot_df['N'].unique())
N_min = plot_df['N'].min()

N_dropdown = widgets.Dropdown(
                            options = N_options,
                            value = N_min,
                            description = 'N',
                            disabled = False
                        )

container = widgets.HBox([widgets.VBox([beta_slider, alpha_slider, K_slider]), N_dropdown])

In [5]:
eps = 0.01

def get_curr_plot_df(plot_df):
    curr_df = (plot_df[
                    plot_df.
                        beta.
                        between(beta_slider.value - eps,
                                beta_slider.value + eps) 
                        &
                    plot_df.
                        alpha.
                        between(alpha_slider.value - eps,
                                alpha_slider.value + eps)
                        &
                    plot_df.
                        K.
                        between(K_slider.value - eps,
                                K_slider.value + eps)
                        &
                    plot_df.
                        N.
                        between(N_dropdown.value - eps,
                                N_dropdown.value + eps)].
                 groupby('degree').
                 mean().
                 reset_index())
    return curr_df

def response(change):
    curr_df = get_curr_plot_df(plot_df)
    with w_fig.batch_update():
        w_fig.data[0].x = curr_df.degree
        w_fig.data[0].y = curr_df.frequency
        w_fig.layout.xaxis.range = (-1, N_dropdown.value)
        w_fig.layout.yaxis.range = (-0.005, 1/np.sqrt(N_dropdown.value))

In [6]:
curr_plot_df = get_curr_plot_df(plot_df)
trace = go.Scatter(x = curr_plot_df.degree, y =  curr_plot_df.frequency, name = 'degree distribution')
w_fig = go.FigureWidget(data = trace,
                          layout = go.Layout(title = {'text' : f'Degree distribution for different network parameters'})
                   )

w_fig.layout.xaxis.title = 'degree'
w_fig.layout.yaxis.title = 'frequency'
w_fig.layout.xaxis.range = (-1, N_dropdown.value)
w_fig.layout.yaxis.range = (-0.005, 1/np.sqrt(N_dropdown.value))

In [7]:
beta_slider.observe(response, 
                    names = "value")
alpha_slider.observe(response, 
                    names = "value")
K_slider.observe(response, 
                    names = "value")

In [8]:
final_widget = widgets.VBox([container, w_fig])
final_widget